# Discovering and Visualizing Topics in Texts

Most typical cases of text classification in NLP (named entity recognition, question answering, etc) require training datasets where each piece of text is associted with a label. However, in real-life scenarios, text collections rarely come with metadata labels that tell you what the texts are about. When people answer open-ended survey questions, for example, they don't repeat detectable keywords in their answer with the topics they discuss.

**Topic modeling** is an unsupervised classification technique that is able to discover the topics in a collection of texts by looking at their commonalities. In this context, "topics" refers to groups of related words that often occur together in the same text. For example, in a collection of newspaper articles a topic model may identify one topic that is made up of words such as "politician", "law", and "parliament", and another characterized by words such as "player", "match" and "penalty". Topic models only go as far as identifying clusters of related words; a human is still needed to interpret these clusters and give them labels such as "politics" and "football". 

One of the most popular topic models is Latent Dirichlet Allocation (LDA). LDA is a generative model that sees every text as a mixture of topics and each sentence as a mixture of words. For example, the "football" topic will generate the word "penalty" with a high probability, while the "politics" topic will have a much higher probability for "politician" than for "penalty". Other words, such as "the" and "an", will have similar probabilities in all topics. LDA takes its name from the Dirichlet probability distribution. This is the prior distribution it assumes the topics in a text will have.

Modified from https://github.com/nlptown/nlp-notebooks/blob/master/Discovering%20and%20Visualizing%20Topics%20in%20Texts%20with%20LDA.ipynb

## Data

Insight fellows frequently come up with project ideas that revolve around topic modeling of online reviews. For this example, we'll use a curated dataset of Amazon reviews for toys and games filtered to products with more than 500 reviews. This dataset was collected and cleaned in the `0.Data Scraping and Prep` notebook.

In [23]:
import pandas as pd
import numpy as np

# This command will unzip the text file
# The rest of the cell will run even if this command fails
!gzip -d ../data/reviews_amazon_small.csv.gz

reviews = pd.read_csv('../data/reviews_amazon_small.csv')

gzip: can't stat: ../data/reviews_amazon_small.csv.gz (../data/reviews_amazon_small.csv.gz.gz): No such file or directory


Each of the rows in this data frame contains a product identifier, a product title, a rating left by a user, the text of their review, and a clean version of the text:

In [24]:
reviews.head()

,product_id,product_title,rating,text,text_processed
0,0975277324,Ticket To Ride,4.0,Why not 5 stars? THis game only got 4 due to t...,why not stars this game only got due to the ...
1,0975277324,Ticket To Ride,5.0,One of the best board games around This is an ...,one of the best board games around this is an ...
2,0975277324,Ticket To Ride,5.0,One of our favorites of all time! What are you...,one of our favorites of all time what are you ...
3,0975277324,Ticket To Ride,5.0,Great Game This is my families first experienc...,great game this is my families first experienc...
4,0975277324,Ticket To Ride,4.0,A Ticket to Fun I didn't think that I was goin...,a ticket to fun i didnt think that i was going...


## Preprocessing

Before we train a topic model, we need to tokenize our texts. Let's do this with the [spaCy](https://spacy.io/) NLP library. We need to load a statisti English and use spaCy to perform our first preprocessing pass:

In [3]:
import spacy

# If you haven't installed the spaCy language model, uncomment the following line and run this cell
# ! python -m spacy download en_core_web_sm

# You will need to restart the notebook (go to the menu Kernel -> Restart) and re-run cells up to this point

In [25]:
nlp = spacy.load('en_core_web_sm')

reviews = reviews.sample(n=2000)

texts = reviews['text_processed'].tolist()
%time spacy_docs = list(nlp.pipe(texts))

CPU times: user 16.9 s, sys: 4.23 s, total: 21.2 s
Wall time: 21.3 s


The text of each review is now a spaCy Doc that we can transform into a list of tokens. Instead of the original tokens, we're going to work with the **lemmas** instead. This will allow our model to generalize and understand that different forms of a word should be treated as one.

Stemming and Lemmatization both generate the root form of the words. Lemmatization uses the rules about a language and the resulting tokens are all actual words. For example, the word "thought" becomes the lemma "think". Stemming is a crude heuristic that chops off the ends of words such that the resulting tokens may not be actual words. Stemming is faster but only works well for simple words like "toys" and "toy".

This is the full list of our initial preprocessing steps: 
 
- remove all words shorter than 2 characters (these are often fairly uninteresting from a topical point of view)
- drop all stopwords
- lowercase remaining lemmas

In [27]:
docs = [[t.lemma_.lower() for t in doc if len(t.orth_) > 2 and not t.is_stop] for doc in spacy_docs]
for i in range(5):
    print(docs[i])
    print('\n')

['hysterical', 'game', 'absolutely', 'hysterical', 'easily', 'embarrassed', 'offend', 'slightly', 'crudetwiste', 'sense', 'humor', 'share', 'like', 'minded', 'individual']


['make', 'cut', 'cut', 'wrapping', 'paper', 'scissor', 'generally', 'annoying', 'process', 'cut', 'cut', 'cut', 'width', 'try', 'perfect', 'angle', 'scissor', 'glide', 'cut', 'paper', '   ', 'paper', 'sort', 'jam', 'tear', 'scissor', 'blade', 'cry', 'tear', 'ruin', 'wrap', 'paper', 'scotch', 'gift', 'wrap', 'cutter', 'right', 'tool', 'job', 'slice', 'paper', 'snags', 'work', 'perfectly', 'cut', 'cleanly', 'go', 'wrap', 'lot', 'gift', 'want', 'hand']


['look', 'like', 'phone', 'app', 'tablet', 'app', 'open', 'boy', 'identify', 'ipad', 'good', 'want', 'look', 'similar', 'use', 'love', 'leappad', 'ask', 'look', 'like', 'phone', 'app', 'tablet', 'app', 'great', 'feel', 'like', 'similar', 'like', 'able', 'explore', 'app', 'like', 'worry', 'mess', 'setting', 'delete', 'thing', 'get', 'want', 'inyou', 'like', 'consider',

Next, we also want to take frequent bigrams into account. **Bigrams are multiword units**, such as "colored pencil" that actually form one word rather than two. We'll use Gensim to first identify the frequent bigrams in the corpus, then append them to the list of tokens for the documents in which they appear. This means the bigrams will not be in their correct position in the text, but that's fine: topic models are bag-of-word models that ignore word position anyway.

In [28]:
import re
from gensim.models import Phrases

bigram = Phrases(docs, min_count=10)
tokens = []

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:  # bigrams can be recognized by the "_" that joins the invidual words
            docs[idx].append(token)
            tokens.append(token)
            
print(list(set(tokens))[:10])

['card_humanity', 'ask_question', 'long_time', 'lose_interest', 'apple_apple', 'waste_money', 'work_fine', 'shape_sorter', 'little_girl', 'perform_action']


Next, we move on to the final Gensim-specific preprocessing steps. First, we create a dictionary representation of the documents. This dictionary will map each word to a unique ID and help us create bag-of-word representations of each document. These bag-of-word representations contain the ids of the words in the document, together with their frequency. Additionally, we can remove the least and most frequent words from the vocabulary. This improves the quality of our topic model and speeds up its training. The minimum frequency of a word is expressed as an absolute number, the maximum frequency is the proportion of documents a word is allowed to occur in.

In [29]:
from gensim.corpora import Dictionary

dictionary = Dictionary(docs)
print('Number of unique words in original documents:', len(dictionary))

dictionary.filter_extremes(no_below=3, no_above=0.25)
print('Number of unique words after removing rare and common words:', len(dictionary))

print("Example representation of document 3:", dictionary.doc2bow(docs[2]))

Number of unique words in original documents: 8392
Number of unique words after removing rare and common words: 2589
Example representation of document 3: [(39, 2), (43, 1), (44, 5), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 3), (58, 1), (59, 1), (60, 2), (61, 1), (62, 2), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1)]


Then we create bag-of-word representations for each document in the corpus:

In [30]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

## Training

Now it's time to train our topic model. We do this with the following parameters: 

- `corpus`: the bag-of-word representations of our documents
- `id2token`: the mapping from indices to words
- `num_topics`: the number of topics we want the model to identify
- `chunksize`: the number of documents the model sees for every update
- `passes`: the number of times we show the total corpus to the model during training
- `random_state`: we use a seed to ensure reproducibility.

On a corpus of this size, the training will typically about a minute.

In [31]:
from gensim.models import LdaModel

%time model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, chunksize=500, passes=3, random_state=1)

CPU times: user 3.51 s, sys: 19.1 ms, total: 3.53 s
Wall time: 3.54 s


## Results

Let's take a look at what the model has learnt. We do this by printing out the ten words that are most characteristic for each of the topics. Most topics show common words like "play", "love" and "game" but it's hard to identify any other patterns in the data.

In [32]:
for (topic, words) in model.print_topics():
    print(topic+1, ":", words, '\n')

1 : 0.026*"battery" + 0.022*"leappad" + 0.015*"app" + 0.011*"time" + 0.011*"buy" + 0.010*"purchase" + 0.010*"use" + 0.009*"moon" + 0.008*"work" + 0.008*"helicopter" 

2 : 0.033*"card" + 0.020*"good" + 0.018*"family" + 0.014*"time" + 0.012*"adult" + 0.012*"young" + 0.012*"board" + 0.011*"child" + 0.011*"simple" + 0.011*"age" 

3 : 0.015*"good" + 0.015*"year_old" + 0.013*"use" + 0.010*"easy" + 0.010*"little" + 0.009*"car" + 0.009*"think" + 0.008*"plastic" + 0.008*"try" + 0.007*"quality" 

4 : 0.027*"elf" + 0.013*"little" + 0.012*"son" + 0.012*"set" + 0.011*"tile" + 0.011*"cute" + 0.011*"boy" + 0.010*"year_old" + 0.010*"book" + 0.010*"grandson" 

5 : 0.022*"month" + 0.018*"son" + 0.016*"letter" + 0.015*"daughter" + 0.015*"ball" + 0.014*"learn" + 0.014*"sound" + 0.013*"buy" + 0.013*"time" + 0.011*"song" 

6 : 0.075*"card" + 0.035*"apple" + 0.034*"action" + 0.034*"player" + 0.018*"person" + 0.016*"expansion" + 0.016*"repeat" + 0.013*"apple_apple" + 0.013*"pete" + 0.012*"friend" 

7 : 0.016*

Another way of inspecting the topics is by visualizing them. This can be done with the [pyLDAvis](https://github.com/bmabey/pyLDAvis) library. PyLDAvis will show us how popular the topics are in our corpus, how similar the topics are, and which are the most salient words for this topic. Note it's important to set `sort_topics=False` on the call to pyLDAvis. If you don't, it will order the topics differently than Gensim. 

In [33]:
import pyLDAvis.gensim
import warnings

pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=False)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.042879  0.110510       1        1  17.174549
1      0.124543  0.033257       2        1  11.620284
2     -0.037853  0.071756       3        1  10.490746
3     -0.057335 -0.168173       4        1   6.071644
4     -0.074178 -0.088128       5        1  17.081936
5      0.326318 -0.035177       6        1   3.569532
6     -0.071344  0.037466       7        1   9.933081
7     -0.046142  0.095109       8        1   8.482985
8     -0.048700  0.066286       9        1   7.971765
9     -0.072429 -0.122907      10        1   7.603473, topic_info=        Term        Freq       Total Category  logprob  loglift
487     card  463.000000  463.000000  Default  30.0000  30.0000
313  battery  318.000000  318.000000  Default  29.0000  29.0000
967   player  174.000000  174.000000  Default  28.0000  28.0000
56   leappad  286.000000  286.000000  Default  27.0000  27.0000
736     book  202.000000  202.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
237     hold   29.168245  166.384109  Topic10  -5.1747   0.8353
240    piece   29.128174  191.318314  Topic10  -5.1760   0.6943
79     think   29.645180  407.933167  Topic10  -5.1584  -0.0452
308  receive   26.497181  108.575729  Topic10  -5.2707   1.1662
349     time   27.137051  634.628540  Topic10  -5.2468  -0.5756

[698 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         1  0.054048  absolutely
0         2  0.072063  absolutely
0         3  0.018016  absolutely
0         4  0.396349  absolutely
0         5  0.306270  absolutely
...     ...       ...         ...
390       8  0.034844       young
390       9  0.014933       young
390      10  0.064710       young
1985      5  0.932845   youngster
422       4  0.865025       zingo

[1754 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

Finally, let's inspect the topics the model recognizes in some of the individual documents. Here we see how LDA tends to assign a high probability to a low number of topics for each documents, which makes its results easily interpretable.

In [34]:
for (text, doc) in zip(texts[:20], docs[:20]):
    print(text)
    print('-'*10)
    print([(topic+1, prob) for (topic, prob) in model[dictionary.doc2bow(doc)] if prob > 0.3])
    print('\n')

hysterical this game is absolutely hysterical if youre not easily embarrassed or offended and have a slightly crudetwisted sense of humor must be shared with like minded individuals
----------
[(2, 0.3344441), (7, 0.60394734)]


it makes the cut cutting wrapping paper with a scissors is generally an annoying process you can either go cut cut cut along the full width or try to get that perfect angle where the scissors glides and cuts across the paper    until it doesnt and then the paper sort of jams up and tears between the scissors blades well cry no more tears over ruined wrapping paper the scotch gift wrap cutter is the right tool for the job it slices across the paper with no snags works perfectly and cuts cleanly if youre going to be wrapping a lot of gifts youll want one of these in your hand
----------
[(8, 0.7474394)]


this looks just like many phones with apps and tablets with apps upon opening it my boy identified it as an ipad which is good we wanted something that looked s

In [35]:
docs

[['hysterical',
  'game',
  'absolutely',
  'hysterical',
  'easily',
  'embarrassed',
  'offend',
  'slightly',
  'crudetwiste',
  'sense',
  'humor',
  'share',
  'like',
  'minded',
  'individual',
  'sense_humor'],
 ['make',
  'cut',
  'cut',
  'wrapping',
  'paper',
  'scissor',
  'generally',
  'annoying',
  'process',
  'cut',
  'cut',
  'cut',
  'width',
  'try',
  'perfect',
  'angle',
  'scissor',
  'glide',
  'cut',
  'paper',
  '   ',
  'paper',
  'sort',
  'jam',
  'tear',
  'scissor',
  'blade',
  'cry',
  'tear',
  'ruin',
  'wrap',
  'paper',
  'scotch',
  'gift',
  'wrap',
  'cutter',
  'right',
  'tool',
  'job',
  'slice',
  'paper',
  'snags',
  'work',
  'perfectly',
  'cut',
  'cleanly',
  'go',
  'wrap',
  'lot',
  'gift',
  'want',
  'hand',
  'gift_wrap'],
 ['look',
  'like',
  'phone',
  'app',
  'tablet',
  'app',
  'open',
  'boy',
  'identify',
  'ipad',
  'good',
  'want',
  'look',
  'similar',
  'use',
  'love',
  'leappad',
  'ask',
  'look',
  'like',


Looping through all texts, let's save the most likely topic number.

In [36]:
topic_nums = []
for (text, doc) in zip(texts, docs):
    probs = np.array(model[dictionary.doc2bow(doc)])
    topic_nums.append(probs[np.argsort(probs[:,-1])][-1,0])
    
reviews['topic'] = topic_nums

product_vs_topic = pd.crosstab(reviews['product_title'], reviews['topic'])
product_vs_topic = product_vs_topic.T / product_vs_topic.sum(axis = 1) * 100
product_vs_topic = product_vs_topic.T

In [37]:
import matplotlib.pyplot as plt
from matplotlib import colors
def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

product_vs_topic.round(2).style.apply(background_gradient,
               cmap='YlGnBu',
               m=product_vs_topic.min().min(),
               M=product_vs_topic.max().max(),
               low=0.5,
               high=0.8)

topic,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
product_title,,,,,,,,,,
Apples to Apples Party Box - The Game of Crazy Combinations (Family Edition),0.000000,53.570000,3.570000,3.570000,10.710000,17.860000,3.570000,7.140000,0.000000,0.000000
Battat Take-A-Part Airplane,6.520000,2.170000,13.040000,2.170000,6.520000,0.000000,52.170000,13.040000,2.170000,2.170000
Campfire Kids Campfire,77.270000,0.000000,4.550000,9.090000,0.000000,0.000000,9.090000,0.000000,0.000000,0.000000
Cards Against Humanity,0.000000,47.620000,7.940000,3.170000,3.170000,30.160000,4.760000,1.590000,0.000000,1.590000
Cards Against Humanity: First Expansion,0.000000,47.370000,0.000000,0.000000,0.000000,42.110000,10.530000,0.000000,0.000000,0.000000
Cards Against Humanity: Second Expansion,0.000000,40.000000,10.000000,0.000000,0.000000,40.000000,5.000000,0.000000,5.000000,0.000000
"Educational Insights The Sneaky, Snacky Squirrel Game",0.000000,14.810000,48.150000,14.810000,7.410000,0.000000,7.410000,0.000000,0.000000,7.410000
Fisher Price Learning Home,4.000000,4.000000,4.000000,4.000000,72.000000,0.000000,0.000000,8.000000,0.000000,4.000000
"Fisher-Price Brilliant Basics Baby""s First Blocks",0.000000,2.700000,8.110000,5.410000,67.570000,0.000000,5.410000,2.700000,2.700000,5.410000


## Conclusions

Many collections of unstructured texts don't come with any labels. Topic models such as Latent Dirichlet Allocation are a useful technique to discover the most prominent topics in such documents. Gensim makes training these topics model easy, and pyLDAvis presents the results in a visually attractive way. Together they form a powerful toolkit to better understand what's inside large sets of documents and to explore subsets of related texts. However, these methods can perform poorly in short texts with vague or unspecified subjects. Although traditional topic models are lacking in more semantic information (they don't use word embeddings, for instance), they can be really quick way of getting insights into large collections of documents.